In [0]:
    

import datetime
import psycopg2
from contextlib import closing
dbinfo = {
    'NAME': 'dailyest',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt


sql = """select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from plproxy.execute_select_nestloop($proxy$ select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from ms.market_size_daily_estimate_1000 where date = '2019-12-01' and category_id in (801) $proxy$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);"""
rows, _ = select(sql, None)
print len(rows)
print rows




In [0]:


import datetime
import psycopg2
from contextlib import closing
dbinfo = {
    'NAME': 'dailyest',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt


sql_1 = """select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from plproxy.execute_select_nestloop($proxy$ select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate from ms.market_size_daily_estimate_1000 where date = '2019-12-01' and category_id = 801$proxy$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT) GROUP BY device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate;"""


# sql = """select * from plproxy.execute_select_nestloop($proxy$ select device_id,count(distinct store_id) from ms.market_size_daily_estimate_1000 where date='2019-12-01' GROUP BY device_id $proxy$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);"""
# sql = """select * from plproxy.execute_select_nestloop($proxy$ select device_id,count(distinct store_id) as store_id_count from ms.market_size_daily_estimate_1000 where date='2019-12-01' AND app_type_id = 0 AND purchase_type_id = 12 AND kpi = 1 AND category_id=600  GROUP BY device_id $proxy$) tbl (device_id SMALLINT, store_id_count BIGINT);"""

sql = """select SUM(estimate) as estimate from plproxy.execute_select_nestloop($proxy$ select estimate from ms.market_size_daily_estimate_1000 where date BETWEEN '2019-12-01' and '2019-12-01' AND category_id=1 AND app_type_id=0 AND purchase_type_id=10 AND kpi=1 AND store_id=10$proxy$) tbl (estimate BIGINT);"""


# actual_sql = """select * from plproxy.execute_select_nestloop($proxy$ select estimate,device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id from ms.market_size_daily_estimate_1000 where date BETWEEN '2019-12-01' and '2019-12-07' AND category_id=1 AND app_type_id=0 AND purchase_type_id=10 AND kpi=1 AND store_id=10 $proxy$) tbl ( estimate BIGINT,device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT);"""




led_test = "select device_id, date from plproxy.execute_select_nestloop($proxy$ select device_id, date from ms.latest_date $proxy$, 0) t (device_id INT, date DATE);"

rows, _ = select(led_test, None)
# rows, _ = select(sql_1, None)

# print len(rows)
print rows




In [0]:

from pyspark.sql.functions import lit, udf
from pyspark.sql.types import (
    ArrayType, BooleanType, LongType, IntegerType,
    StringType, StructType, StructField
)
schema = StructType([
    StructField("store_id", IntegerType(), False),
    StructField("date", StringType(), False),
    StructField("platform_id", IntegerType(), False),
    StructField("device", StringType(), False),
    StructField("data_type", StringType(), False),
    StructField("price_type", IntegerType(), False),
    StructField("purchase_type", IntegerType(), False),
    StructField("category_id", IntegerType(), False),
    StructField("estimate", LongType(), False)
])
spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-01/android/market_size/10", schema=schema, sep='\t').filter("price_type=0 and purchase_type=10").createOrReplaceTempView("raw_data")

res = spark.sql("select SUM(revenue) AS total from raw_data PIVOT (AVG(estimate) for data_type in ('downloads', 'revenue'))").first()
print res[0]


In [0]:

from pyspark.sql.functions import lit, udf
from pyspark.sql.types import (
    ArrayType, BooleanType, LongType, IntegerType,
    StringType, StructType, StructField
)
schema = StructType([
    StructField("store_id", IntegerType(), False),
    StructField("date", StringType(), False),
    StructField("platform_id", IntegerType(), False),
    StructField("device", StringType(), False),
    StructField("data_type", StringType(), False),
    StructField("price_type", IntegerType(), False),
    StructField("purchase_type", IntegerType(), False),
    StructField("category_id", IntegerType(), False),
    StructField("estimate", LongType(), False)
])
spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-01/ios/", schema=schema, sep='\t').show()



In [0]:


import datetime
import psycopg2
from contextlib import closing
dbinfo = {
    'NAME': 'dailyest',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

# should return 65
gp_country_count = """select COUNT( store_id)from plproxy.execute_select_nestloop($proxy$ select store_id from ms.market_size_daily_estimate where date='2019-12-01' AND device_id=1000$proxy$) tbl (store_id INT);"""

# should return 156
ios_country_count ="""select COUNT(distinct store_id)from plproxy.execute_select_nestloop($proxy$ select store_id from ms.market_size_daily_estimate where date='2019-12-01' AND device_id in(2000)$proxy$) tbl (store_id INT);"""

#86
ios_category_count ="""select SUM(cnt) from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt from ms.market_size_daily_estimate_2000 where date='2019-12-01' AND store_id=143441$proxy$) tbl (cnt BIGINT);"""

#103
gp_category_count ="""select SUM(cnt)from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt from ms.market_size_daily_estimate_1000 where date='2019-12-01' AND store_id=10$proxy$) tbl (cnt BIGINT);"""

rows, _ = select(gp_category_count, None)
print rows[0][0]

In [0]:


spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-01/android/market_size/10", schema=schema, sep='\t').filter("category_id='1' and purchase_type='10' and price_type='0'").createOrReplaceTempView("raw_data")
spark.sql("select * from raw_data PIVOT (AVG(estimate) for data_type in ('downloads', 'revenue'))").show()


In [0]:


import datetime
import psycopg2
from contextlib import closing
dbinfo = {
    'NAME': 'dailyest',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

# should return 65
gp_country_count = """select COUNT(distinct store_id)from plproxy.execute_select_nestloop($proxy$ select store_id from ms.market_size_daily_estimate_1000 where date='2019-12-01'$proxy$) tbl (store_id INT);"""

# should return 156
ios_country_count ="""select COUNT(distinct store_id)from plproxy.execute_select_nestloop($proxy$ select store_id from ms.market_size_daily_estimate where date='2019-12-01' AND device_id in(2000)$proxy$) tbl (store_id INT);"""

#86
ios_category_count ="""select COUNT(distinct category_id)from plproxy.execute_select_nestloop($proxy$ select category_id from ms.market_size_daily_estimate_2000 where date='2019-12-01' AND store_id=143441$proxy$) tbl (category_id INT);"""

#103
gp_category_count ="""select COUNT(distinct category_id)from plproxy.execute_select_nestloop($proxy$ select category_id from ms.market_size_daily_estimate_1000 where date='2019-12-01' AND store_id=10$proxy$) tbl (category_id INT);"""


# Test Metric Values

#PLproxy this have category filter
downloads = """select SUM(estimate) as estimate from plproxy.execute_select_nestloop($proxy$ select estimate from ms.market_size_daily_estimate_1000 where date BETWEEN '2019-12-01' and '2019-12-01' AND app_type_id=0 AND purchase_type_id=10 AND kpi=1 AND store_id=10$proxy$) tbl (estimate BIGINT);"""
revenue = """select SUM(estimate) as estimate from plproxy.execute_select_nestloop($proxy$ select estimate from ms.market_size_daily_estimate_1000 where date BETWEEN '2019-12-01' and '2019-12-01'  AND app_type_id=0 AND purchase_type_id=10 AND kpi=2 AND store_id=10$proxy$) tbl (estimate BIGINT);"""
rows, _ = select(revenue, None)
print rows[0][0]

rows, _ = select(downloads, None)
print rows[0][0]
# print len(rows)

#should be the same value with below
from pyspark.sql.functions import lit, udf
from pyspark.sql.types import (
    ArrayType, BooleanType, LongType, IntegerType,
    StringType, StructType, StructField
)
schema = StructType([
    StructField("store_id", IntegerType(), False),
    StructField("date", StringType(), False),
    StructField("platform_id", IntegerType(), False),
    StructField("device", StringType(), False),
    StructField("data_type", StringType(), False),
    StructField("price_type", IntegerType(), False),
    StructField("purchase_type", IntegerType(), False),
    StructField("category_id", IntegerType(), False),
    StructField("estimate", LongType(), False)
])
spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-01/android/market_size/10", schema=schema, sep='\t').filter("price_type=0 and purchase_type=10").createOrReplaceTempView("raw_data")
revenue = spark.sql("select SUM(revenue) AS total from raw_data PIVOT (AVG(estimate) for data_type in ('downloads', 'revenue'))").first()
print revenue[0]
download = spark.sql("select SUM(downloads) AS total from raw_data PIVOT (AVG(estimate) for data_type in ('downloads', 'revenue'))").first()
print download[0]





In [0]:
%%sh


aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date=2019-12-01/

In [0]:

from pyspark.sql.functions import sum as spark_sum
df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact/month=2019-12/device_id=2000/").filter("store_id=0 and app_type_id=0 and purchase_type_id=10 and kpi=1 and date='2019-12-01' and category_id in(36)").select("estimate").groupBy().sum().collect()
print(df)

In [0]:


from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings




def pg_settings(urn, sql, granularity, schema):
    result = {}
    db_settings = build_db_settings(urn, schema)
    connection = ClusterConnection(db_settings)
    master_runner = connection.master_runner
    rows, _ = master_runner.select(sql)
    result[granularity] = rows
    return result

urn = Urn(
    namespace='app-qa.db-check.v1',
    owner='app_qa'
)





# test country count
gp_country_count = """select sum(cnt) from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt from ms.market_size_daily_estimate_1000 where date='2019-12-08'$proxy$) tbl (cnt BIGINT);"""
res = pg_settings(urn, gp_country_count, '2019-12-01', 'daily_est')
print res['2019-12-01'][0][0]
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact//month=2019-12/device_id=1000/").filter("date='2019-12-01'").count()

# test category count
# gp_category_count ="""select SUM(cnt)from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt from ms.market_size_daily_estimate_1000 where date='2019-12-01' AND store_id=10$proxy$) tbl (cnt BIGINT);"""
# res = pg_settings(urn, gp_category_count, '2019-12-01', 'daily_est')
# print res['2019-12-01'][0][0]
# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact//month=2019-12/device_id=1000/").filter("date='2019-12-01' and store_id=10").count()






In [0]:



from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings




def pg_settings(urn, sql, granularity, schema):
    result = {}
    db_settings = build_db_settings(urn, schema)
    connection = ClusterConnection(db_settings)
    master_runner = connection.master_runner
    rows, _ = master_runner.select(sql)
    result[granularity] = rows
    return result

urn = Urn(
    namespace='app-qa.db-check.v1',
    owner='app_qa'
)
# test country count
gp_country_count = """select sum(cnt) from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt from ms.market_size_daily_estimate_1000 where date='2019-12-01'$proxy$) tbl (cnt BIGINT);"""

# res = pg_settings(urn, gp_country_count, '2019-12-01', 'daily_est')
# print res['2019-12-01'][0][0]

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact//month=2019-12/device_id=1000/").filter("date='2019-12-01'").count()

# test category count
# gp_category_count ="""select SUM(cnt)from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt from ms.market_size_daily_estimate_1000 where date='2019-12-01' AND store_id=10$proxy$) tbl (cnt BIGINT);"""
# res = pg_settings(urn, gp_category_count, '2019-12-01', 'daily_est')
# print res['2019-12-01'][0][0]
UC = [600, 601, 801, 802, 803, 808, 804, 805, 806, 816, 807, 809, 810, 811, 812, 813, 814, 815, 602, 701, 702, 703, 704, 705, 721, 706, 708, 722, 709, 710, 712, 713, 714, 715, 716, 717, 718, 719, 720, 711, 723, 707, 901, 902, 903]

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact//month=2019-12/device_id=1000/").filter("app_type_id=0 and purchase_type_id=10 and date='2019-12-01' and kpi=1 and store_id=10 and category_id not in (600, 601, 801, 802, 803, 808, 804, 805, 806, 816, 807, 809, 810, 811, 812, 813, 814, 815, 602, 701, 702, 703, 704, 705, 721, 706, 708, 722, 709, 710, 712, 713, 714, 715, 716, 717, 718, 719, 720, 711, 723, 707, 901, 902, 903)").select("category_id","store_id").collect().

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date=2019-12-01/").filter("country_code='WW' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all' ").show()


new = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date=2019-12-01/").filter("country_code='WW' and app_price_type_id=0 and purchase_type_id=10 and device_code='ios-all' ").select("est_market_size_download").groupBy().sum().collect()
print(new)



old=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact/month=2019-12/device_id=2000/").filter("store_id=0 and app_type_id=0 and purchase_type_id=10 and kpi=1 and date='2019-12-01' and category_id not in(600, 601, 801, 802, 803, 808, 804, 805, 806, 816, 807, 809, 810, 811, 812, 813, 814, 815, 602, 701, 702, 703, 704, 705, 721, 706, 708, 722, 709, 710, 712, 713, 714, 715, 716, 717, 718, 719, 720, 711, 723, 707, 901, 902, 903)").select("estimate").groupBy().sum().collect()
print(old)




In [0]:

import os
import shutil
import unittest

from conf.settings import ROOT
from aadatapipelinecore.core.fs.device import meta_bucket, raw_bucket, unified_bucket
from aadatapipelinecore.core.fs.device.bucket import unified_data_system_config_bucket
from aadatapipelinecore.core.utils.commandline import env
from aadatapipelinecore.core.utils.encode import activate_system_utf8
from aadatapipelinecore.core.utils.spark import create_spark, eject_all_caches


class PySparkTest(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        """Setup a basic Spark context for testing"""
        activate_system_utf8()
        env(PYTHONIOENCODING='utf8')
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext
        cls._release_resource()

    @classmethod
    def tearDownClass(cls):
        cls._release_resource()
        cls.sc = None
        # comment below code for performance
        # cls.spark.stop()

    @classmethod
    def _release_resource(cls):
        # clear spark cached and persisted files
        eject_all_caches(cls.spark)
        # clear physical files
        cls._empty_bucket(meta_bucket())
        cls._empty_bucket(raw_bucket())
        cls._empty_bucket(unified_bucket())
        cls._empty_bucket(unified_data_system_config_bucket())
        cls._empty_spark_db_dir("{}/metastore_db".format(ROOT))
        cls._empty_spark_db_dir("{}/spark-warehouse".format(ROOT))

    @classmethod
    def _empty_bucket(cls, bucket):
        try:
            if os.environ.get('EMPTY_BUCKET', "true").lower() != "false":
                bucket.empty()
        except OSError:
            pass

    @classmethod
    def _empty_spark_db_dir(cls, db_dir):
        try:
            shutil.rmtree(db_dir, ignore_errors=False)
        except OSError:
            pass
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import random
from applications.db_check_v1.common.constants import pg_settings, COUNTRY_MAPPING_BY_MARKET
from applications.db_check_v1.common.table_common_info import urn
from applications.db_check_v1.common.utils import get_date_list, string_to_datetime

DUMP_FILE_PATH = \
    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact/" \
    "month={}-{}/device_id={}/"
APP_PRICE_TYPE_ID = {'ALL': 0, 'FREE': 1, 'PAID': 2}
PURCHASE_TYPE_ID = {'ALL': 10, 'TRUE': 11, 'FALSE': 12}
UNIFIED_FILE_PATH = ""
def check_country_count(date, device_id):
        year = string_to_datetime(date).year
        
        month = string_to_datetime(date).month
        if month<10:
            month = '0'+str(month)
        sql = """select SUM(cnt)from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt 
        from ms.market_size_daily_estimate 
        where date='{date}' AND device_id={device_id}$proxy$) tbl (cnt BIGINT);""".format(date=date,
                                                                                          device_id=device_id)
        pg_count = pg_settings(urn, sql, date, 'daily_est')[date][0][0]

        parquet_count = spark.read.parquet(DUMP_FILE_PATH.format(year, month, device_id)).filter(
            "date='{}'".format(date)).count()
        assert(pg_count==parquet_count),"can not match {} - {}".format(date, device_id)
def test_gp_country_count():
    for date in random.sample(get_date_list("2013-01-01", "2020-01-01"), 50):
        print "Running:"+str(date)
        check_country_count(date, 1000)


def check_category_count(date, device_id, store_id):
        year = string_to_datetime(date).year
        month = string_to_datetime(date).month
        if month<10:
            month = '0'+str(month)
        sql = """select SUM(cnt)from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt from 
        ms.market_size_daily_estimate_{device_id} 
        where date='{date}' AND store_id={store_id}$proxy$) tbl (cnt BIGINT);""".format(device_id=device_id,
                                                                                        date=date,
                                                                                        store_id=store_id)
        pg_count = pg_settings(urn, sql, date, "daily_est")[date][0][0]

        parquet_count = spark.read.parquet(DUMP_FILE_PATH.format(year, month, device_id)).filter(
            "store_id={} and date='{}'".format(store_id, date)).count()

        assert(pg_count==parquet_count),"can not match {} - {} - {}".format(date, device_id, store_id)

def test_gp_category_count():
    for date in random.sample(get_date_list("2013-01-01", "2020-01-01"), 50):
        for store_id in COUNTRY_MAPPING_BY_MARKET['gp']:
            print "{}-{}".format(date, store_id)
            check_category_count(date, 1000, store_id)

test_gp_category_count()


In [0]:

import os
import shutil
import unittest

from conf.settings import ROOT
from aadatapipelinecore.core.fs.device import meta_bucket, raw_bucket, unified_bucket
from aadatapipelinecore.core.fs.device.bucket import unified_data_system_config_bucket
from aadatapipelinecore.core.utils.commandline import env
from aadatapipelinecore.core.utils.encode import activate_system_utf8
from aadatapipelinecore.core.utils.spark import create_spark, eject_all_caches


class PySparkTest(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        """Setup a basic Spark context for testing"""
        activate_system_utf8()
        env(PYTHONIOENCODING='utf8')
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext
        cls._release_resource()

    @classmethod
    def tearDownClass(cls):
        cls._release_resource()
        cls.sc = None
        # comment below code for performance
        # cls.spark.stop()

    @classmethod
    def _release_resource(cls):
        # clear spark cached and persisted files
        eject_all_caches(cls.spark)
        # clear physical files
        cls._empty_bucket(meta_bucket())
        cls._empty_bucket(raw_bucket())
        cls._empty_bucket(unified_bucket())
        cls._empty_bucket(unified_data_system_config_bucket())
        cls._empty_spark_db_dir("{}/metastore_db".format(ROOT))
        cls._empty_spark_db_dir("{}/spark-warehouse".format(ROOT))

    @classmethod
    def _empty_bucket(cls, bucket):
        try:
            if os.environ.get('EMPTY_BUCKET', "true").lower() != "false":
                bucket.empty()
        except OSError:
            pass

    @classmethod
    def _empty_spark_db_dir(cls, db_dir):
        try:
            shutil.rmtree(db_dir, ignore_errors=False)
        except OSError:
            pass
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import random
from applications.db_check_v1.common.constants import pg_settings, COUNTRY_MAPPING_BY_MARKET
from applications.db_check_v1.common.table_common_info import urn
from applications.db_check_v1.common.utils import get_date_list, string_to_datetime

DUMP_FILE_PATH = \
    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact/" \
    "month={}-{}/device_id={}/"
APP_PRICE_TYPE_ID = {'ALL': 0, 'FREE': 1, 'PAID': 2}
PURCHASE_TYPE_ID = {'ALL': 10, 'TRUE': 11, 'FALSE': 12}
UNIFIED_FILE_PATH = ""
def check_country_count(date, device_id):
        year = string_to_datetime(date).year
        
        month = string_to_datetime(date).month
        if month<10:
            month = '0'+str(month)
        sql = """select SUM(cnt)from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt 
        from ms.market_size_daily_estimate 
        where date='{date}' AND device_id={device_id}$proxy$) tbl (cnt BIGINT);""".format(date=date,
                                                                                          device_id=device_id)
        pg_count = pg_settings(urn, sql, date, 'daily_est')[date][0][0]

        parquet_count = spark.read.parquet(DUMP_FILE_PATH.format(year, month, device_id)).filter(
            "date='{}'".format(date)).count()
        assert(pg_count==parquet_count),"can not match {} - {}".format(date, device_id)
def test_gp_country_count():
    for date in random.sample(get_date_list("2013-01-01", "2020-01-01"), 50):
        print "Running:"+str(date)
        check_country_count(date, 1000)


def check_category_count(date, device_id, store_id):
        year = string_to_datetime(date).year
        month = string_to_datetime(date).month
        if month<10:
            month = '0'+str(month)
        sql = """select SUM(cnt)from plproxy.execute_select_nestloop($proxy$ select count(1) as cnt from 
        ms.market_size_daily_estimate_{device_id} 
        where date='{date}' AND store_id={store_id}$proxy$) tbl (cnt BIGINT);""".format(device_id=device_id,
                                                                                        date=date,
                                                                                        store_id=store_id)
        pg_count = pg_settings(urn, sql, date, "daily_est")[date][0][0]

        parquet_count = spark.read.parquet(DUMP_FILE_PATH.format(year, month, device_id)).filter(
            "store_id={} and date='{}'".format(store_id, date)).count()

        assert(pg_count==parquet_count),"can not match {} - {} - {}".format(date, device_id, store_id)

def test_gp_category_count():
    for date in random.sample(get_date_list("2013-01-01", "2020-01-01"), 50):
        for store_id in COUNTRY_MAPPING_BY_MARKET['gp']:
            print "{}-{}".format(date, store_id)
            check_category_count(date, 1000, store_id)
            
def check_metric_value(date, device_id, store_id, kpi):

        year = string_to_datetime(date).year
        month = string_to_datetime(date).month
        if month<10:
            month = '0'+str(month)
        sql = """select SUM(estimate) as estimate from plproxy.execute_select_nestloop
        ($proxy$ select estimate from ms.market_size_daily_estimate_{device_id} 
        where date='{date}' AND app_type_id={app_price_type_id}
        AND purchase_type_id={purchase_type_id} AND kpi={kpi} AND store_id={store_id}$proxy$) tbl (estimate BIGINT);""".format(
            device_id=device_id, date=date, app_price_type_id=APP_PRICE_TYPE_ID['ALL'],
            purchase_type_id=PURCHASE_TYPE_ID['ALL'], kpi=kpi, store_id=store_id)

        pg_metric_value = pg_settings(urn, sql, date, "daily_est")[date][0][0]
        parquet_metric_value = spark.read.parquet(DUMP_FILE_PATH.format(year, month, device_id)).filter(
            "store_id={store_id} and app_type_id={app_type_id} "
            "and purchase_type_id={purchase_type_id} and kpi={kpi} and date='{date}'".format(store_id=store_id,
                                                                                             app_type_id=APP_PRICE_TYPE_ID['ALL'],
                                                                                             purchase_type_id=PURCHASE_TYPE_ID['ALL'],
                                                                                             kpi=kpi,
                                                                                             date=date)).select(
            "estimate").groupBy().sum().collect()[0][0]
        assert(pg_metric_value==parquet_metric_value),"can not match {}-{}-{}-{}".format(date, device_id, store_id, kpi)           
def test_ios_metric_data_accuracy():
        KPI = [1, 2]
        for date in random.sample(get_date_list("2013-01-01", "2020-01-01"), 50):
            for store_id in COUNTRY_MAPPING_BY_MARKET['ios']:
                print "{}-{}".format(date, store_id)
                kpi = random.choice(KPI)
                check_metric_value(date, 2000, store_id, kpi)

test_ios_metric_data_accuracy()


In [0]:

import os
import shutil
import unittest
import random
import time
from conf.settings import ROOT
from aadatapipelinecore.core.fs.device import meta_bucket, raw_bucket, unified_bucket
from aadatapipelinecore.core.fs.device.bucket import unified_data_system_config_bucket
from aadatapipelinecore.core.utils.commandline import env
from aadatapipelinecore.core.utils.encode import activate_system_utf8
from aadatapipelinecore.core.utils.spark import create_spark, eject_all_caches

DUMP_FILE_PATH = \
    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact/" \
    "month={}-{}/device_id={}/"

UNIFIED_FILE_PATH = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/" \
                    "granularity=daily/date={}/"
class PySparkTest(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        """Setup a basic Spark context for testing"""
        activate_system_utf8()
        env(PYTHONIOENCODING='utf8')
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext
        cls._release_resource()

    @classmethod
    def tearDownClass(cls):
        cls._release_resource()
        cls.sc = None
        # comment below code for performance
        # cls.spark.stop()

    @classmethod
    def _release_resource(cls):
        # clear spark cached and persisted files
        eject_all_caches(cls.spark)
        # clear physical files
        cls._empty_bucket(meta_bucket())
        cls._empty_bucket(raw_bucket())
        cls._empty_bucket(unified_bucket())
        cls._empty_bucket(unified_data_system_config_bucket())
        cls._empty_spark_db_dir("{}/metastore_db".format(ROOT))
        cls._empty_spark_db_dir("{}/spark-warehouse".format(ROOT))

    @classmethod
    def _empty_bucket(cls, bucket):
        try:
            if os.environ.get('EMPTY_BUCKET', "true").lower() != "false":
                bucket.empty()
        except OSError:
            pass

    @classmethod
    def _empty_spark_db_dir(cls, db_dir):
        try:
            shutil.rmtree(db_dir, ignore_errors=False)
        except OSError:
            pass
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
COUNTRY_MAPPING_BY_MARKET = {
    'gp': {1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB',
           8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA',
           15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID',
           22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL',
           29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT',
           36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO',
           43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE',
           50: 'KW', 51: 'SA', 52: 'CO', 53: 'KZ', 54: 'PK', 56: 'PE', 61: 'AZ',
           62: 'EC', 64: 'CR', 65: 'LB', 73: 'QA', 74: 'NG', 78: 'LT', 80: 'HR',
           86: 'LV', 95: 'KE', 1000: 'WW'},
    'ios': {143441: 'US', 143442: 'FR', 143443: 'DE', 143444: 'GB', 143445: 'AT',
            143446: 'BE', 143447: 'FI', 143448: 'GR', 143449: 'IE', 143450: 'IT',
            143451: 'LU', 143452: 'NL', 143453: 'PT', 143454: 'ES', 143455: 'CA',
            143456: 'SE', 143457: 'NO', 143458: 'DK', 143459: 'CH', 143460: 'AU',
            143461: 'NZ', 143462: 'JP', 143463: 'HK', 143464: 'SG', 143465: 'CN',
            143466: 'KR', 143467: 'IN', 143468: 'MX', 143469: 'RU', 143470: 'TW',
            143471: 'VN', 143472: 'ZA', 143473: 'MY', 143474: 'PH', 143475: 'TH',
            143476: 'ID', 143477: 'PK', 143478: 'PL', 143479: 'SA', 143480: 'TR',
            143481: 'AE', 143482: 'HU', 143483: 'CL', 143484: 'NP', 143485: 'PA',
            143486: 'LK', 143487: 'RO', 143489: 'CZ', 143491: 'IL', 143492: 'UA',
            143493: 'KW', 143494: 'HR', 143495: 'CR', 143496: 'SK', 143497: 'LB',
            143498: 'QA', 143499: 'SI', 143501: 'CO', 143502: 'VE', 143503: 'BR',
            143504: 'GT', 143505: 'AR', 143506: 'SV', 143507: 'PE', 143508: 'DO',
            143509: 'EC', 143510: 'HN', 143511: 'JM', 143512: 'NI', 143513: 'PY',
            143514: 'UY', 143515: 'MO', 143516: 'EG', 143517: 'KZ', 143518: 'EE',
            143519: 'LV', 143520: 'LT', 143521: 'MT', 143523: 'MD', 143524: 'AM',
            143525: 'BW', 143526: 'BG', 143528: 'JO', 143529: 'KE', 143530: 'MK',
            143531: 'MG', 143532: 'ML', 143533: 'MU', 143534: 'NE', 143535: 'SN',
            143536: 'TN', 143537: 'UG', 143538: 'AI', 143539: 'BS', 143540: 'AG',
            143541: 'BB', 143542: 'BM', 143543: 'VG', 143544: 'KY', 143545: 'DM',
            143546: 'GD', 143547: 'MS', 143548: 'KN', 143549: 'LC', 143550: 'VC',
            143551: 'TT', 143552: 'TC', 143553: 'GY', 143554: 'SR', 143555: 'BZ',
            143556: 'BO', 143557: 'CY', 143558: 'IS', 143559: 'BH', 143560: 'BN',
            143561: 'NG', 143562: 'OM', 143563: 'DZ', 143564: 'AO', 143565: 'BY',
            143566: 'UZ', 143568: 'AZ', 143571: 'YE', 143572: 'TZ', 143573: 'GH',
            143575: 'AL', 143576: 'BJ', 143577: 'BT', 143578: 'BF', 143579: 'KH',
            143580: 'CV', 143581: 'TD', 143582: 'CG', 143583: 'FJ', 143584: 'GM',
            143585: 'GW', 143586: 'KG', 143587: 'LA', 143588: 'LR', 143589: 'MW',
            143590: 'MR', 143591: 'FM', 143592: 'MN', 143593: 'MZ', 143594: 'NA',
            143595: 'PW', 143597: 'PG', 143598: 'ST', 143599: 'SC', 143600: 'SL',
            143601: 'SB', 143602: 'SZ', 143603: 'TJ', 143604: 'TM', 143605: 'ZW',
            0: 'WW'},
}
DEVICE_ID_TO_DEVICE_CODE_MAPPING = {
    1000: 'android-all',
    2000: 'ios-all',
    2001: 'ios-phone',
    2002: 'ios-tablet'
}
DEVICE_ID_TYPE_MAPPING = {
    1000: 'gp',
    2000: 'ios',
    2001: 'ios',
    2002: 'ios'
}
UC = [600, 601, 801, 802, 803, 808, 804, 805, 806, 816, 807, 809, 810, 811, 812, 813, 814, 815, 602, 701, 702, 703, 704, 705, 721, 706, 708, 722, 709, 710, 712, 713, 714, 715, 716, 717, 718, 719, 720, 711, 723, 707, 901, 902, 903]
APP_PRICE_TYPE_ID = {'ALL': 0, 'FREE': 1, 'PAID': 2}
PURCHASE_TYPE_ID = {'ALL': 10, 'TRUE': 11, 'FALSE': 12}
def _parse_date(date):
        year = string_to_datetime(date).year
        month = string_to_datetime(date).month
        if month < 10:
            month = '0' + str(month)

        return year, month
        




# new = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date=2019-12-01/").filter("country_code='WW' and app_price_type_id=0 and purchase_type_id=10 and device_code='ios-all' ").select("est_market_size_download").groupBy().sum().collect()
# print(new)



# old=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact/month=2019-12/device_id=2000/").filter("store_id=0 and app_type_id=0 and purchase_type_id=10 and kpi=1 and date='2019-12-01' and category_id not in(600, 601, 801, 802, 803, 808, 804, 805, 806, 816, 807, 809, 810, 811, 812, 813, 814, 815, 602, 701, 702, 703, 704, 705, 721, 706, 708, 722, 709, 710, 712, 713, 714, 715, 716, 717, 718, 719, 720, 711, 723, 707, 901, 902, 903)").select("estimate").groupBy().sum().collect()
# print(old)



    
def get_dump_estimate(date, device_id, store_id, kpi):
        year, month = _parse_date(date)
        dump_estimate = spark.read.parquet(DUMP_FILE_PATH.format(year, month, device_id)).filter(
            "store_id={store_id} and app_type_id={app_type_id} "
            "and purchase_type_id={purchase_type_id} and kpi={kpi} and date='{date}' and category_id not in(600, 601, 801, 802, 803, 808, 804, 805, 806, 816, 807, 809, 810, 811, 812, 813, 814, 815, 602, 701, 702, 703, 704, 705, 721, 706, 708, 722, 709, 710, 712, 713, 714, 715, 716, 717, 718, 719, 720, 711, 723, 707, 901, 902, 903)".format(store_id=store_id,
                                                                                             app_type_id=
                                                                                             APP_PRICE_TYPE_ID['ALL'],
                                                                                             purchase_type_id=
                                                                                             PURCHASE_TYPE_ID['ALL'],
                                                                                             kpi=kpi,
                                                                                             date=date)).select(
            "estimate").groupBy().sum().collect()[0][0]
        return dump_estimate

def get_unified_estimate(date, device_code, country_code, metric):
        unified_estimate = spark.read.parquet(UNIFIED_FILE_PATH.format(date)).filter(
            "country_code='{country_code}' and app_price_type_id={app_price_type_id} "
            "and purchase_type_id={purchase_type_id} and device_code='{device_code}' ".format(country_code=country_code,
                                                                                              app_price_type_id=
                                                                                              APP_PRICE_TYPE_ID['ALL'],
                                                                                              purchase_type_id=
                                                                                              PURCHASE_TYPE_ID['ALL'],
                                                                                              device_code=device_code,
                                                                                              )).select(
            "{metric}".format(metric=metric)).groupBy().sum().collect()[0][0]
        return unified_estimate

def check_metric_value(date, device_id, store_id, kpi):
        KPI_TO_METRIC = {1: 'est_market_size_download', 2: 'est_market_size_revenue'}
        dump_metric_value = get_dump_estimate(date, device_id, store_id, kpi)
        device_code = DEVICE_ID_TO_DEVICE_CODE_MAPPING[device_id]
        # ios or gp
        device_type = DEVICE_ID_TYPE_MAPPING[device_id]
        country_code = COUNTRY_MAPPING_BY_MARKET[device_type][store_id]
        unified_metric_value = get_unified_estimate(date, device_code, country_code, KPI_TO_METRIC[kpi])
        assert(dump_metric_value==unified_metric_value),"can not match {}-{}-{}-{}".format(date, device_id, store_id, kpi)






def test_ios_data_accuracy():
        KPI = [1, 2]
        for date in random.sample(get_date_list("2013-01-01", "2020-01-01"), 200):
            start_time = time.time()
            for store_id in COUNTRY_MAPPING_BY_MARKET['ios']:
                print "{} - {}".format(date, store_id)
                kpi = random.choice(KPI)
                check_metric_value(date, 2002, store_id, kpi)
            end_time = time.time()
            print "############################# one day cost :{}".format(end_time - start_time)
test_ios_data_accuracy()


In [0]:

from pyspark.sql.functions import lit, udf
from pyspark.sql.types import (
    ArrayType, BooleanType, LongType, IntegerType,
    StringType, StructType, StructField
)
schema = StructType([
    StructField("store_id", IntegerType(), False),
    StructField("date", StringType(), False),
    StructField("platform_id", IntegerType(), False),
    StructField("device", StringType(), False),
    StructField("data_type", StringType(), False),
    StructField("price_type", IntegerType(), False),
    StructField("purchase_type", IntegerType(), False),
    StructField("category_id", IntegerType(), False),
    StructField("estimate", LongType(), False)
])

# Rountine unified data
# new = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date=2019-12-01/").filter("country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='ios-all' ").select("est_market_size_download").groupBy().sum().collect()
# print new
# # raw data 





# # Rountine RAW Data
# # store_id|      date|platform_id|     device|data_type|price_type|purchase_type|category_id|estimate
# spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-01/ios/market_size/143441", schema=schema, sep='\t').filter("price_type=0 and purchase_type=10 and data_type='downloads' and category_id not in (600, 601, 801, 802, 803, 808, 804, 805, 806, 816, 807, 809, 810, 811, 812, 813, 814, 815, 602, 701, 702, 703, 704, 705, 721, 706, 708, 722, 709, 710, 712, 713, 714, 715, 716, 717, 718, 719, 720, 711, 723, 707, 901, 902, 903) and device='ios'").select("estimate").groupBy().sum().collect()









spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date=2020-01-10/").filter("country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all' and category_id=400002").show()
spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-10/android/market_size/10", schema=schema, sep='\t').filter("price_type=0 and purchase_type=10 and data_type='downloads' and category_id=38 and device='google-play'").show()




























In [0]:

from pyspark.sql.functions import lit, udf
from pyspark.sql.types import (
    ArrayType, BooleanType, LongType, IntegerType,
    StringType, StructType, StructField
)
schema = StructType([
    StructField("store_id", IntegerType(), False),
    StructField("date", StringType(), False),
    StructField("platform_id", IntegerType(), False),
    StructField("device", StringType(), False),
    StructField("data_type", StringType(), False),
    StructField("price_type", IntegerType(), False),
    StructField("purchase_type", IntegerType(), False),
    StructField("category_id", IntegerType(), False),
    StructField("estimate", LongType(), False)
])


# Rountine unified data
# new = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date=2020-01-10/").filter("country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all' and category_id=400001").show()
# print new
# raw data 

# Rountine RAW Data
# store_id|      date|platform_id|     device|data_type|price_type|purchase_type|category_id|estimate
# spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-10/android/market_size/10", schema=schema, sep='\t').filter("price_type=0 and purchase_type=10 and data_type='downloads' and category_id=2 and device='google-play'").show()




spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date=2020-01-10/").filter("country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all' and category_id=400002").show()





spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-10/android/market_size/10", schema=schema, sep='\t').filter("price_type=0 and purchase_type=10 and data_type='downloads' and category_id=38 and device='google-play'").show()


























In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
select * from store.store_market_size_fact_v1 where date='2020-01-11' and country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all' and category_id=400001;
EOF

In [0]:


# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.market-size.v1/fact/granularity=daily/date=2020-01-11/").filter("country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all'").select("est_market_size_download").groupBy().sum().collect()

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/date=2020-01-11/").filter("country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all'").select("est_market_size_download").groupBy().sum().collect()














In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/granularity=daily/

In [0]:

import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
from aaplproxy.connection import ClusterConnection
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
def citus_settings(schema):
    template = "CITUS_" + schema.upper() + "_CITUS_DB_{property}"
    host, port = getattr(settings, template.format(property='HOSTS'))[0]
    return (
        "dbname='{db}' user='{user}' password='{password}' "
        "host='{host}' port='{port}'".format(
            db=getattr(settings, template.format(property='NAME')),
            user=getattr(settings, template.format(property='ACCESS_ID')),
            host=host,
            password=getattr(settings, template.format(property='SECRET_KEY')),
            port=port
        )
    )

sql = "select sum(est_market_size_download) from store.store_market_size_fact_v1 where date='2020-01-11' and country_code='US' and app_price_type_id=0 and purchase_type_id=10 and device_code='android-all' and category_id=400001;"
result = query(citus_settings("aa"), sql)
print result[0][0]

In [0]:

import os
import shutil
import unittest
import random
import time
from conf.settings import ROOT
from aadatapipelinecore.core.fs.device import meta_bucket, raw_bucket, unified_bucket
from aadatapipelinecore.core.fs.device.bucket import unified_data_system_config_bucket
from aadatapipelinecore.core.utils.commandline import env
from aadatapipelinecore.core.utils.encode import activate_system_utf8
from aadatapipelinecore.core.utils.spark import create_spark, eject_all_caches
import psycopg2
from applications.db_check_v1.common.utils import get_date_list, string_to_datetime

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
from aaplproxy.connection import ClusterConnection
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
def citus_settings(schema):
    template = "CITUS_" + schema.upper() + "_CITUS_DB_{property}"
    host, port = getattr(settings, template.format(property='HOSTS'))[0]
    return (
        "dbname='{db}' user='{user}' password='{password}' "
        "host='{host}' port='{port}'".format(
            db=getattr(settings, template.format(property='NAME')),
            user=getattr(settings, template.format(property='ACCESS_ID')),
            host=host,
            password=getattr(settings, template.format(property='SECRET_KEY')),
            port=port
        )
    )
    
DUMP_FILE_PATH = \
    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.legacy-market_size_daily_estimate.v2/fact/" \
    "month={}-{}/device_id={}/"

UNIFIED_FILE_PATH = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/app-tech.store.market-size.v1/fact/" \
                    "granularity=daily/date={}/"
class PySparkTest(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        """Setup a basic Spark context for testing"""
        activate_system_utf8()
        env(PYTHONIOENCODING='utf8')
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext
        cls._release_resource()

    @classmethod
    def tearDownClass(cls):
        cls._release_resource()
        cls.sc = None
        # comment below code for performance
        # cls.spark.stop()

    @classmethod
    def _release_resource(cls):
        # clear spark cached and persisted files
        eject_all_caches(cls.spark)
        # clear physical files
        cls._empty_bucket(meta_bucket())
        cls._empty_bucket(raw_bucket())
        cls._empty_bucket(unified_bucket())
        cls._empty_bucket(unified_data_system_config_bucket())
        cls._empty_spark_db_dir("{}/metastore_db".format(ROOT))
        cls._empty_spark_db_dir("{}/spark-warehouse".format(ROOT))

    @classmethod
    def _empty_bucket(cls, bucket):
        try:
            if os.environ.get('EMPTY_BUCKET', "true").lower() != "false":
                bucket.empty()
        except OSError:
            pass

    @classmethod
    def _empty_spark_db_dir(cls, db_dir):
        try:
            shutil.rmtree(db_dir, ignore_errors=False)
        except OSError:
            pass
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
COUNTRY_MAPPING_BY_MARKET = {
    'gp': {1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB',
           8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA',
           15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID',
           22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL',
           29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT',
           36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO',
           43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE',
           50: 'KW', 51: 'SA', 52: 'CO', 53: 'KZ', 54: 'PK', 56: 'PE', 61: 'AZ',
           62: 'EC', 64: 'CR', 65: 'LB', 73: 'QA', 74: 'NG', 78: 'LT', 80: 'HR',
           86: 'LV', 95: 'KE', 1000: 'WW'},
    'ios': {143441: 'US', 143442: 'FR', 143443: 'DE', 143444: 'GB', 143445: 'AT',
            143446: 'BE', 143447: 'FI', 143448: 'GR', 143449: 'IE', 143450: 'IT',
            143451: 'LU', 143452: 'NL', 143453: 'PT', 143454: 'ES', 143455: 'CA',
            143456: 'SE', 143457: 'NO', 143458: 'DK', 143459: 'CH', 143460: 'AU',
            143461: 'NZ', 143462: 'JP', 143463: 'HK', 143464: 'SG', 143465: 'CN',
            143466: 'KR', 143467: 'IN', 143468: 'MX', 143469: 'RU', 143470: 'TW',
            143471: 'VN', 143472: 'ZA', 143473: 'MY', 143474: 'PH', 143475: 'TH',
            143476: 'ID', 143477: 'PK', 143478: 'PL', 143479: 'SA', 143480: 'TR',
            143481: 'AE', 143482: 'HU', 143483: 'CL', 143484: 'NP', 143485: 'PA',
            143486: 'LK', 143487: 'RO', 143489: 'CZ', 143491: 'IL', 143492: 'UA',
            143493: 'KW', 143494: 'HR', 143495: 'CR', 143496: 'SK', 143497: 'LB',
            143498: 'QA', 143499: 'SI', 143501: 'CO', 143502: 'VE', 143503: 'BR',
            143504: 'GT', 143505: 'AR', 143506: 'SV', 143507: 'PE', 143508: 'DO',
            143509: 'EC', 143510: 'HN', 143511: 'JM', 143512: 'NI', 143513: 'PY',
            143514: 'UY', 143515: 'MO', 143516: 'EG', 143517: 'KZ', 143518: 'EE',
            143519: 'LV', 143520: 'LT', 143521: 'MT', 143523: 'MD', 143524: 'AM',
            143525: 'BW', 143526: 'BG', 143528: 'JO', 143529: 'KE', 143530: 'MK',
            143531: 'MG', 143532: 'ML', 143533: 'MU', 143534: 'NE', 143535: 'SN',
            143536: 'TN', 143537: 'UG', 143538: 'AI', 143539: 'BS', 143540: 'AG',
            143541: 'BB', 143542: 'BM', 143543: 'VG', 143544: 'KY', 143545: 'DM',
            143546: 'GD', 143547: 'MS', 143548: 'KN', 143549: 'LC', 143550: 'VC',
            143551: 'TT', 143552: 'TC', 143553: 'GY', 143554: 'SR', 143555: 'BZ',
            143556: 'BO', 143557: 'CY', 143558: 'IS', 143559: 'BH', 143560: 'BN',
            143561: 'NG', 143562: 'OM', 143563: 'DZ', 143564: 'AO', 143565: 'BY',
            143566: 'UZ', 143568: 'AZ', 143571: 'YE', 143572: 'TZ', 143573: 'GH',
            143575: 'AL', 143576: 'BJ', 143577: 'BT', 143578: 'BF', 143579: 'KH',
            143580: 'CV', 143581: 'TD', 143582: 'CG', 143583: 'FJ', 143584: 'GM',
            143585: 'GW', 143586: 'KG', 143587: 'LA', 143588: 'LR', 143589: 'MW',
            143590: 'MR', 143591: 'FM', 143592: 'MN', 143593: 'MZ', 143594: 'NA',
            143595: 'PW', 143597: 'PG', 143598: 'ST', 143599: 'SC', 143600: 'SL',
            143601: 'SB', 143602: 'SZ', 143603: 'TJ', 143604: 'TM', 143605: 'ZW',
            0: 'WW'},
}
DEVICE_ID_TO_DEVICE_CODE_MAPPING = {
    1000: 'android-all',
    2000: 'ios-all',
    2001: 'ios-phone',
    2002: 'ios-tablet'
}
DEVICE_ID_TYPE_MAPPING = {
    1000: 'gp',
    2000: 'ios',
    2001: 'ios',
    2002: 'ios'
}
UC = [600, 601, 801, 802, 803, 808, 804, 805, 806, 816, 807, 809, 810, 811, 812, 813, 814, 815, 602, 701, 702, 703, 704, 705, 721, 706, 708, 722, 709, 710, 712, 713, 714, 715, 716, 717, 718, 719, 720, 711, 723, 707, 901, 902, 903]
APP_PRICE_TYPE_ID = {'ALL': 0, 'FREE': 1, 'PAID': 2}
PURCHASE_TYPE_ID = {'ALL': 10, 'TRUE': 11, 'FALSE': 12}
def _parse_date(date):
        year = string_to_datetime(date).year
        month = string_to_datetime(date).month
        if month < 10:
            month = '0' + str(month)

        return year, month
        

def get_estimate_from_citus(date, device_code, country_code, metric):
    sql = "select sum({metric}) from store.store_market_size_fact_v1 where date='{date}' and country_code='{country_code}' and app_price_type_id=0 and purchase_type_id=10 and device_code='{device_code}';".format(metric=metric, date=date,country_code=country_code,device_code=device_code)
    result = query(citus_settings("aa"), sql)
    return result[0][0]
    



def get_unified_estimate(date, device_code, country_code, metric):
        unified_estimate = spark.read.parquet(UNIFIED_FILE_PATH.format(date)).filter(
            "country_code='{country_code}' and app_price_type_id={app_price_type_id} "
            "and purchase_type_id={purchase_type_id} and device_code='{device_code}' ".format(country_code=country_code,
                                                                                              app_price_type_id=
                                                                                              APP_PRICE_TYPE_ID['ALL'],
                                                                                              purchase_type_id=
                                                                                              PURCHASE_TYPE_ID['ALL'],
                                                                                              device_code=device_code,
                                                                                              )).select(
            "{metric}".format(metric=metric)).groupBy().sum().collect()[0][0]
        return unified_estimate

def check_metric_value(date, device_id, store_id, kpi):
        KPI_TO_METRIC = {1: 'est_market_size_download', 2: 'est_market_size_revenue'}
        device_code = DEVICE_ID_TO_DEVICE_CODE_MAPPING[device_id]
        # ios or gp
        device_type = DEVICE_ID_TYPE_MAPPING[device_id]
        country_code = COUNTRY_MAPPING_BY_MARKET[device_type][store_id]
        citus_value = get_estimate_from_citus(date, device_code, country_code, KPI_TO_METRIC[kpi])
        unified_metric_value = get_unified_estimate(date, device_code, country_code, KPI_TO_METRIC[kpi])
        #  print(citus_value, unified_metric_value)
        assert(citus_value==unified_metric_value),"can not match {}-{}-{}-{}".format(date, device_id, store_id, kpi)






def test_ios_data_accuracy():
        KPI = [1, 2]
        for date in get_date_list("2010-07-04", "2020-01-11"):
            start_time = time.time()
            for store_id in COUNTRY_MAPPING_BY_MARKET['ios']:
                print "{} - {}".format(date, store_id)
                kpi = random.choice(KPI)
                check_metric_value(date, 2002, store_id, kpi)
            end_time = time.time()
            print "############################# one day cost :{}".format(end_time - start_time)
test_ios_data_accuracy()


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
select sum(estimate) from plproxy.execute_select_nestloop(\$proxy\$ 
    select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate
    from ms.market_size_daily_estimate_1000 
    where 
        date = '2019-12-08' and
        purchase_type_id=10 and 
        app_type_id=0 and 
        store_id  in(8, 80, 16, 54, 47, 46, 48, 28, 36, 15, 4, 30, 95, 56, 40, 53, 23, 20, 44, 25, 1, 26, 86, 49, 13, 22, 45, 27, 21, 5, 43, 11, 39, 3, 61, 14, 17, 50, 33, 19, 51, 31, 65, 35, 52, 37, 34, 32, 12, 10, 42, 18, 78, 9, 24, 64, 84, 38, 74, 6, 29, 2, 41, 7) and 
        category_id in (1) and
        device_id in(1000) and
        kpi=1
\$proxy\$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);
EOF

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
select distinct(store_id) from plproxy.execute_select_nestloop(\$proxy\$ 
    select device_id, date, store_id, kpi, category_id, app_type_id, purchase_type_id, estimate
    from ms.market_size_daily_estimate_1000 
    where 
        date = '2019-12-08' and
        purchase_type_id=10 and 
        app_type_id=0 and 
        category_id in (1) and
        device_id in(1000) and
        kpi=1
\$proxy\$) tbl (device_id SMALLINT, date DATE, store_id INT, kpi SMALLINT, category_id INT, app_type_id SMALLINT, purchase_type_id SMALLINT, estimate BIGINT);
EOF

In [0]:
%%sh


PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
SELECT sum(est_market_size_download) AS est_market_size_download, sum(est_market_size_revenue) as est_market_size_revenue FROM store.store_market_size_fact_v1 WHERE app_price_type_id=0 AND purchase_type_id=10 AND device_code in ('android-all') AND country_code not in ('WW') AND category_id in (400000) AND date between '2019-12-08' AND '2019-12-14' ORDER BY est_market_size_download desc;
EOF


In [0]:
%%sh


PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
SELECT sum(est_market_size_download) AS est_market_size_download, sum(est_market_size_revenue) as est_market_size_revenue,country_code FROM store.store_market_size_fact_v1 WHERE app_price_type_id=0 AND purchase_type_id=10 AND device_code in ('android-all') AND country_code not in ('WW') AND category_id in (400000) AND date between '2019-12-08' AND '2019-12-14' group by country_code;
EOF



In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF  
SELECT distinct(country_code) FROM store.store_market_size_fact_v1 WHERE app_price_type_id=0 AND purchase_type_id=10 AND device_code in ('android-all') AND country_code not in ('WW') AND category_id in (400000) AND date between '2019-12-08' AND '2019-12-14';
EOF

In [0]:

from pyspark.sql.functions import lit, udf
from pyspark.sql.types import (
    ArrayType, BooleanType, LongType, IntegerType,
    StringType, StructType, StructField
)
schema = StructType([
    StructField("store_id", IntegerType(), False),
    StructField("date", StringType(), False),
    StructField("platform_id", IntegerType(), False),
    StructField("device", StringType(), False),
    StructField("data_type", StringType(), False),
    StructField("price_type", IntegerType(), False),
    StructField("purchase_type", IntegerType(), False),
    StructField("category_id", IntegerType(), False),
    StructField("estimate", LongType(), False)
])
ww_res = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-08/android/market_size/1000", schema=schema, sep='\t').filter("price_type=0 and purchase_type=10 and category_id=1").collect()
store_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61,62, 64, 65, 73, 74, 78, 80, 86, 95]
pl_s=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 61, 64, 65, 74, 78, 80, 84, 86, 95]
pl_ww = [1000]

s3_store_list = [1,10,1003,1004,1005,1006,1007,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,36,37,38,39,4,40,41,42,43,44,45,46,47,48,49,5,50,51,52,53,54,56,6,61,64,65,7,74,78,8,80,84,86,9,95
]
print(ww_res)
# Row(store_id=1000, date=u'2019-12-08', platform_id=0, device=u'google-play', data_type=u'revenue', price_type=0, purchase_type=10, category_id=1, estimate=57864205)
# Row(store_id=1000, date=u'2019-12-08', platform_id=0, device=u'google-play', data_type=u'downloads', price_type=0, purchase_type=10, category_id=1, estimate=207141484)
total_estimate = []
for store_id in pl_s:
    res = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-08/android/market_size/{}".format(store_id), schema=schema, sep='\t').filter("price_type=0 and purchase_type=10 and category_id=1").collect()
    for _r in res:
        if _r[4]=='downloads':
            total_estimate.append(_r[8])
print sum(total_estimate)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-08/android/market_size/

In [0]:

ios_store = [143441, 143442, 143443, 143444, 143445, 143446, 143447, 143448, 143449, 143450, 143451, 143452, 143453, 143454, 143455, 143456, 143457, 143458, 143459, 143460, 143461, 143462, 143463, 143464, 143465, 143466, 143467, 143468, 143469, 143470, 143471, 143472, 143473, 143474, 143475, 143476, 143477, 143478, 143479, 143480, 143481, 143482, 143483, 143484, 143485, 143486, 143487, 143489, 143491, 143492, 143493, 143494, 143495, 143496, 143497, 143498, 143499, 143501, 143502, 143503, 143504, 143505, 143506, 143507, 143508, 143509, 143510, 143511, 143512, 143513, 143514, 143515, 143516, 143517, 143518, 143519, 143520, 143521, 143523, 143524, 143525, 143526, 143528, 143529, 143530, 143531, 143532, 143533, 143534, 143535, 143536, 143537, 143538, 143539, 143540, 143541, 143542, 143543, 143544, 143545, 143546, 143547, 143548, 143549, 143550, 143551, 143552, 143553, 143554, 143555, 143556, 143557, 143558, 143559, 143560, 143561, 143562, 143563, 143564, 143565, 143566, 143568, 143571, 143572, 143573, 143575, 143576, 143577, 143578, 143579, 143580, 143581, 143582, 143583, 143584, 143585, 143586, 143587, 143588, 143589, 143590, 143591, 143592, 143593, 143594, 143595, 143597, 143598, 143599, 143600, 143601, 143602, 143603, 143604, 143605]

total_estimate = []
for store_id in [0]:
    res = spark.read.csv("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-12-08/ios/market_size/{}".format(store_id), schema=schema, sep='\t').filter("price_type=0 and purchase_type=10 and category_id=36").collect()
    for _r in res:
        if _r[4]=='downloads':
            total_estimate.append(_r[8])
print sum(total_estimate)
# 183239466